## Read train and Validation data

In [1]:
# import resources for handling dataset
from utils.LIPDataset import LIPDataset, RandomCrop, RandomHorizontalFlip, Resize, plot_data, ToTensor
from torchvision import transforms




In [2]:
# read dataset and provide augmentation
# TRAIN DATA
train_transform = transforms.Compose([
                Resize(256),
                RandomCrop(224),
                RandomHorizontalFlip(),
                ToTensor()
            ])

train_data = LIPDataset('dataset/train_data.ftr', 'dataset/train_set', train_transform)

In [3]:
# resource to display from dataset
from utils.LIPDataset import untransform_n_display

In [7]:
# visualize a sample
untransform_n_display(train_data, 5000)

In [8]:
# VALIDATION DATA
val_transform = transforms.Compose([
                        Resize(224),
                        ToTensor()
                        ])

val_data = LIPDataset('dataset/val_data.ftr', 'dataset/val_set', val_transform)

In [12]:
# visualize few sample
untransform_n_display(val_data, 6001)

In [16]:
# length of both
len(train_data), len(val_data), train_data[5]['image'].size(), train_data[6]['keypoints'].size()

(30462, 10000, torch.Size([3, 224, 224]), torch.Size([32]))

### Dataloader and batch size

In [17]:
from torch.utils.data import DataLoader

In [18]:
batch_size = 8

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)

valid_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)

In [26]:
samples = next(iter(train_loader))

In [31]:
samples['keypoints'].size() # batches are formed

torch.Size([8, 32])

## Load model. Define lose function and optimizer